In [85]:
import numpy as np

a = 0.7 #probability that user picks a recommendation, given that he continues
q = 0.3 #probability that the user exits (terminates) the session

K = 4 #num of videos in platform (represents the states - numbered from 0 to K-1)
N = 2  #num of recommendations
u_min = 0.3 #relevance threshold

U = np.random.rand(K, K) #relevance matrix
np.fill_diagonal(U, 0)
U_bool = U > u_min #0 if j is irrelevant to i, 1 if j is relevant to i

C = np.random.randint(0, 2, size=K) #c[i] denotes wheter state i is cached or no.

# for testing purposes:
# temp = [[0, 0.8, 0, 0.8],
#          [0.1, 0, 0.8, 0.8],
#          [0.5, 0.7, 0, 0],
#          [0, 0.8, 0, 0.8]]
# U = np.array(temp)
# temp = [0, 1, 1, 0]
# C = np.array(temp)
# U_bool = U > u_min #0 if j is irrelevant to i, 1 if j is relevant to i

action_set = []
for i in range(K):
    for j in range(K):
        if(i == j):
            continue
        action = [i, j]
        action_set.append(action)


#env model given a state and action returns p(s', r | s, a) for each (s', r) that has prob > 0.
#Then this can be used for the Bellman update
def environment_model(state, action): #state is an integer, action is an N-tuple of recommendations
    scenarios = []
    if(all_relevant(state, action)):
        
        prob = (1-q)*(a/N + (1-a)/K) #probability to choose any recommended video
        for item in action:
            reward = is_cached(item)
            t = (prob, item, reward, False)
            scenarios.append(t)
        
        prob = (1-q)*((1-a)/K) #probability to choose any not recommended video
        for item in range(K):
            if(item in action):
                continue
            reward = is_cached(item)
            t = (prob, item, reward, False)
            scenarios.append(t)
            
    else:
        prob = (1-q)*((1-a)/K) #probability to choose any not recommended video
        for item in range(K):
            reward = is_cached(item)
            t = (prob, item, reward, False)
            scenarios.append(t)
                            
    t = (q, 0, 0, True) #terminal state, with probability q, terminate session
    scenarios.append(t)
    return scenarios
    
    
def is_cached(item):
    return C[item]

def all_relevant(s, w):
    for item in w:
        if(not U_bool[s, item]): #if at least one of recommendations is not relevant return False
            return False
    return True
        
    
def policy_evaluation(pi, gamma = 1.0, epsilon = 1e-10):  #inputs: (1) policy to be evaluated, (2) model of the environment (transition probabilities, etc., see previous cell), (3) discount factor (with default = 1), (4) convergence error (default = 10^{-10})
    prev_V = np.zeros(K) # use as "cost-to-go", i.e. for V(s')
    while True: #performing iterations
        V = np.zeros(K) # current value function to be learnerd
        for s in range(K):  # do for every state
            for prob, next_state, reward, done in environment_model(s, pi[s, :]):  # calculate one Bellman step --> i.e., sum over all probabilities of transitions and reward for that state, the action suggested by the (fixed) policy, the reward earned (dictated by the model), and the cost-to-go from the next state (which is also decided by the model)
                V[s] += prob * (reward + gamma * prev_V[next_state] * (not done))
        if np.max(np.abs(prev_V - V)) < epsilon: #check if the new V estimate is close enough to the previous one; 
            break # if yes, finish loop
        prev_V = V.copy() #freeze the new values (to be used as the next V(s'))
    return V
    
    
def policy_improvement(V, gamma=1.0):  # takes a value function (as the cost to go V(s')), a model, and a discount parameter
    Q = np.zeros((K, len(action_set)), dtype=np.float64) #create a Q value array
    for s in range(K):        # for every state in the environment/model
        for i,a in enumerate(action_set):  # and for every action in that state
            for prob, next_state, reward, done in environment_model(s, a):  #evaluate the action value based on the model and Value function given (which corresponds to the previous policy that we are trying to improve) 
                Q[s][i] += prob * (reward + gamma * V[next_state] * (not done))
#     new_pi = lambda s: {s:a for s, a in enumerate(np.argmax(Q, axis=1))}[s]  # this basically creates the new (improved) policy by choosing at each state s the action a that has the highest Q value (based on the Q array we just calculated)
    new_pi = generate_improved_policy(Q)
    
    return new_pi


def policy_iteration(gamma = 1.0, epsilon = 1e-10):
    t = 0
    pi = generate_random_policy()
    
    while True:
        old_pi = pi  #keep the old policy to compare with new
        V = policy_evaluation(pi, gamma, epsilon)   #evaluate latest policy --> you receive its converged value function
        pi = policy_improvement(V,gamma)          #get a better policy using the value function of the previous one just calculated 
        
        t += 1
    
        if(np.array_equal(old_pi, pi)):# you have converged to the optimal policy if the "improved" policy is exactly the same as in the previous step
            break
    print('converged after %d iterations' %t) #keep track of the number of (outer) iterations to converge
    return V,pi


def generate_random_policy():
    pi = np.zeros((K, N), dtype=np.int8)
    for s in range(K):
        action = np.random.choice(K, size=N, replace=False)
        pi[s, :] = action
    return pi

def generate_improved_policy(Q):
    new_pi = np.zeros((K, N), dtype=np.int8)
    action_idxs = np.argmax(Q, axis=1) 
    for s in range(K):
        best_action_idx = np.argmax(Q[s])
        new_pi[s, :] = action_set[best_action_idx]
    return new_pi


V,pi = policy_iteration()
print(f"\nValue function: {V}\n\n")
print(f"Optimal Policy:\n{pi}")



converged after 2 iterations

Value function: [2.15833333 2.15833333 2.15833333 2.15833333]


Optimal Policy:
[[2 3]
 [2 3]
 [1 3]
 [1 2]]


In [80]:
K = 4
N = 2
pi = np.zeros((K, N), dtype=np.int8)

print(pi)
a = np.random.choice(K, size=N, replace=False)
print(type(a))
pi[0, :] = a
print(pi)
print(a)


# for row in enumerate(range(K)):
#     action = np.random.choice(K, size=N, replace=False)
#     pi[row, :] = action
        
# print(pi)
# print('\n')
# print(pi[0, :])

[[0 0]
 [0 0]
 [0 0]
 [0 0]]
<class 'numpy.ndarray'>
[[3 1]
 [0 0]
 [0 0]
 [0 0]]
[3 1]
